In [1]:
import * as hardhat from 'hardhat'
import '@nomiclabs/hardhat-ethers'
import { ethers } from 'hardhat'
import {smock} from '@defi-wonderland/smock'

import * as bn from './utils/bn'
import * as evm from './utils/evm'
import * as wallet from './utils/wallet'

import {getMainnetSdk} from '@dethcrypto/eth-sdk-client';

In [2]:
// FORK
let blockNumber = 16000000

await evm.reset({
    jsonRpcUrl: process.env['MAINNET_RPC'],
    blockNumber
})

const provider = ethers.provider

In [3]:
const signer = await ethers.getSigner()
const signerAddress = await signer.getAddress()

In [4]:
await wallet.fund(signer.address, bn.toUnit(100))

In [5]:
const gameAbi = require('./out/ButtPlugWars.sol/ButtPlugWars.json')
const gameFactory = await ethers.getContractFactory(gameAbi.abi, gameAbi.bytecode.object, signer)
// const game = await gameFactory.deploy({gasLimit: 10e6})

In [6]:
const FIVE_OUT_OF_NINE = '0xB543F9043b387cE5B3d1F0d916E42D8eA2eBA2E0';
const WETH_9 = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2';
const KEEP3R = '0xeb02addCfD8B773A5FFA6B9d1FE99c566f8c44CC';
const KP3R_LP = '0x3f6740b5898c5D3650ec6eAce9a649Ac791e44D7';
const SUDOSWAP_FACTORY = '0xb16c1342E617A5B6E4b631EB114483FDB289c0A4';
const SUDOSWAP_XYK_CURVE = '0x7942E264e21C5e6CbBA45fe50785a15D3BEb1DA0';


const game = await gameFactory.deploy(FIVE_OUT_OF_NINE, WETH_9, KEEP3R, KP3R_LP, SUDOSWAP_FACTORY, SUDOSWAP_XYK_CURVE)

In [7]:
const FIVEOUTOFNINE_WHALE = '0xC5233C3b46C83ADEE1039D340094173f0f7c1EcF'
const whale = await wallet.impersonate(FIVEOUTOFNINE_WHALE)
const KEEPER_ADDRESS = '0x9429cd74a3984396f3117d51cde46ea8e0e21487'
const keeper = await wallet.impersonate(KEEPER_ADDRESS)

In [8]:
const {chess, sudoPool} = getMainnetSdk(signer)
const officialSudoPool = sudoPool.attach(await game.SUDOSWAP_POOL())

In [9]:
await evm.advanceTimeAndBlock(86400*10)
await game.startEvent();
'game-start'

game-start


In [10]:
await chess.connect(whale).transferFrom(whale._address, signer.address, 180)
await chess.connect(whale).transferFrom(whale._address, signer.address, 181)
await chess.setApprovalForAll(game.address, true)
await game.mintPlayerBadge(180, 0, {value: bn.toUnit(1)})
await game.mintPlayerBadge(181, 1, {value: bn.toUnit(1)})
'player mint'

player mint


In [11]:
await evm.advanceTimeAndBlock(14 * 86400)
await game.pushLiquidity()
'push-liquidity'

push-liquidity


In [18]:
atob((await game.tokenURI(0)).substring(29,1000))

0
{"name" : "ButtPlugWars Scoreboard", "description" : "Scoreboard Badge 
0(0) - 0(0) 
```
6 â â â â â â
5 â â â â â â
4 Â· Â· Â· Â· Â· Â·
3 Â· Â· Â· Â· Â· Â·
2 â â â â â â
1 â â â â â â
  a b c d e f
```", "image_data" : "", "attributes" : [{"trait_type" : "game-score", "value" : "0(0) - 0(0)"}, {"trait_type" : "weight", "value" : 2000}]}


In [29]:
atob((await game.tokenURI(1)).substring(29,1000))

1
{"name" : "Player", "description" : "Player Badge with FiveOutOfNine#180", "image_data" : "", "attributes" : [{"trait_type" : "team", "value" : "ZERO"}, {"trait_type" : "weight", "value" : 1000}, {"trait_type" : "score", "value" : 0}, {"trait_type" : "vote", "value" : "0xfc5246b92cbd8b716d3afbe2be2f7550f707ee3a"}, {"trait_type" : "bonded_token", "value" : "180"}]}


In [23]:
const buttPlugAbi = require('./out/IGame.sol/IButtPlug.json')
const fakeButtPlug = await smock.fake(buttPlugAbi.abi)

In [24]:
await game['getBadgeId(uint256)'](2)

BigNumber { _hex: '0x0100000002', _isBigNumber: true }


In [25]:
fakeButtPlug.owner.returns(signer.address)
await game.mintButtPlugBadge(fakeButtPlug.address)

{
  hash: '0x76a36c614684e04aee532a6ef992786f784768e140cb54e8205f2f4e6628312d',
  type: 2,
  accessList: [],
  blockHash: '0x3e0d9bc063579ac8ee69e854dc5c697f72b914c714d27a7eb02fc40e11a96603',
  blockNumber: 16000011,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x0101a575c1', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x01c7b02182', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad9d8', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 355,
  data: '0x067e5741000000000000000000000000fc5246b92cbd8b716d3afbe2be2f7550f707ee3a',
  r: '0x10e00d77a76adfecd77fda1bd6648b460c3cb2a544566747c5920e255d02d898',
  s: '0x59aeab2b9371b10b0e9e9c2cefbb6ab614f3685934a19d575fc20f9e34caccdd',
  v: 0,
  creates: null,
  chainId: 31337,


In [26]:
// await game.mintButtPlugBadge(fakeButtPlug.address)
await game['voteButtPlug(address,uint256[])'](fakeButtPlug.address, [1, '0x0100000002']);
// await game.voteButtPlug(fakeButtPlug.address, '0x0800000000000002');
'vote fake buttplug'

vote fake buttplug


In [27]:
await chess.connect(whale).transferFrom(whale._address, keeper._address, 182)
'add nft to keeper'

add nft to keeper


In [28]:
await chess.board()

BigNumber {
  _hex: '0x03256230011111100000000000000000099999900bcdecb000000001',
  _isBigNumber: true
}


In [30]:
await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((10 << 6) | 25)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((13 << 6) | 30)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((20 << 6) | 28)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

// reverts
await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((12 << 6) | 28)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

await evm.advanceTimeAndBlock(5*86400)
fakeButtPlug.readMove.returns((28 << 6) | 42)
await game.connect(keeper).executeMove()
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

59295000000000000
29647500000000000
19764999999999999
19764999999999999
19764999999999999


In [31]:
// can buy 3 minted tokens
await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())
await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())
await officialSudoPool.swapTokenForAnyNFTs(1, bn.toUnit(1), signer.address, false, signer.address, {value: bn.toUnit(1)})
console.log((await officialSudoPool.getBuyNFTQuote(1)).inputAmount.toString())

39529999999999999
118589999999999998
0


In [32]:
await game.tokenURI(0)

0
data:application/json;base64,eyJuYW1lIiA6ICJCdXR0UGx1Z1dhcnMgU2NvcmVib2FyZCIsICJkZXNjcmlwdGlvbiIgOiAiU2NvcmVib2FyZCBCYWRnZSAKMCgtMykgLSAwKC0yKSAKYGBgCjYg4pmcIOKZnSDimZsg4pmaIOKZnSDimZwKNSDimZ8gwrcg4pmfIMK3IOKZnyDimZ8KNCDCtyDCtyDCtyDimZ8gwrcgwrcKMyDimZ8gwrcg4pmZIMK3IMK3IOKZmAoyIOKZmSDimZkgwrcg4pmZIOKZmSDimZkKMSDimZYgwrcg4pmVIOKZlCDCtyDimZYKICBhIGIgYyBkIGUgZgpgYGAiLCAiaW1hZ2VfZGF0YSIgOiAiIiwgImF0dHJpYnV0ZXMiIDogW3sidHJhaXRfdHlwZSIgOiAiZ2FtZS1zY29yZSIsICJ2YWx1ZSIgOiAiMCgtMykgLSAwKC0yKSJ9LCB7InRyYWl0X3R5cGUiIDogIndlaWdodCIsICJ2YWx1ZSIgOiAyMDAwfV19


In [34]:
JSON.parse(atob((await game.tokenURI(1)).substring(29,1000)))

1
{
  name: 'Player',
  description: 'Player Badge with FiveOutOfNine#180',
  image_data: '',
  attributes: [
    { trait_type: 'team', value: 'ZERO' },
    { trait_type: 'weight', value: 1000 },
    { trait_type: 'score', value: -3000 },
    {
      trait_type: 'vote',
      value: '0xfc5246b92cbd8b716d3afbe2be2f7550f707ee3a'
    },
    { trait_type: 'bonded_token', value: '180' }
  ]
}


In [35]:
JSON.parse(atob((await game.tokenURI('0x0100000002')).substring(29,1000)))

4294967298
{
  name: 'Player',
  description: 'Player Badge with FiveOutOfNine#181',
  image_data: '',
  attributes: [
    { trait_type: 'team', value: 'ONE' },
    { trait_type: 'weight', value: 1000 },
    { trait_type: 'score', value: -2000 },
    {
      trait_type: 'vote',
      value: '0xfc5246b92cbd8b716d3afbe2be2f7550f707ee3a'
    },
    { trait_type: 'bonded_token', value: '181' }
  ]
}


In [36]:
const chessAbi = require('./eth-sdk/abis/mainnet/chess.json')
const fakeChess = await smock.fake(chessAbi, {address: chess.address})

In [37]:
const NEW_BOARD = '0x03256230011111100000000000000000099999900bcdecb000000001'
const WHITE_CAP = '0x03256230011111100000000000000000099909900bcdecb000000001'
const BLACK_CAP = '0x03256230011011100000000000000000099999900bcdecb000000001'
const BOTH_CAPS = '0x03256230011011100000000000000000099909900bcdecb000000001'

In [38]:
fakeChess.board.reset();
fakeChess.balanceOf.returns(9);
fakeChess.board.returnsAtCall(0, NEW_BOARD)
fakeChess.board.returnsAtCall(1, WHITE_CAP)
await evm.advanceTimeAndBlock(5*86400)
await game.connect(keeper).executeMove()

{
  hash: '0xc3d866512add59cd2641e7af0f579a8a6c3e103419d7f5286ff41deeb094cd80',
  type: 2,
  accessList: [],
  blockHash: '0x8c7b62ba9a7181d9dea80bd218494c98c6585ecc79435d4f1686de9b5965cd20',
  blockNumber: 16000028,
  transactionIndex: 0,
  confirmations: 1,
  from: '0x9429cd74A3984396f3117d51cde46ea8e0e21487',
  gasPrice: BigNumber { _hex: '0x51b4a21c', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x67ce7a38', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 507,
  data: '0x30e6d8b2',
  r: '0x0000000000000000000000000000000000000000000000000000000000000001',
  s: '0x0000000000000000000000000000000000000000000000000000000000000002',
  v: 1,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}


In [39]:
const buttPlugBadge = await game['getBadgeId(address)'](fakeButtPlug.address)

In [40]:
fakeChess.board.returns(NEW_BOARD)
fakeButtPlug.readMove.returns((10 << 6) | 25)

In [41]:
JSON.parse(await game.tokenURI(buttPlugBadge))

26572543991925446583788356340741684019669414790681052197791144607744


SyntaxError: Unexpected token d in JSON at position 0
    at JSON.parse (<anonymous>)
    at evalmachine.<anonymous>:3:27
    at processTicksAndRejections (node:internal/process/task_queues:96:5)
    at runNextTicks (node:internal/process/task_queues:65:3)
    at listOnTimeout (node:internal/timers:528:9)
    at processTimers (node:internal/timers:502:7)
    at async Object.execute (/opt/homebrew/lib/node_modules/tslab/dist/executor.js:175:17)
    at async JupyterHandlerImpl.handleExecuteImpl (/opt/homebrew/lib/node_modules/tslab/dist/jupyter.js:219:18)
    at async JupyterHandlerImpl.handleExecute (/opt/homebrew/lib/node_modules/tslab/dist/jupyter.js:177:21)
    at async ZmqServer.handleExecute (/opt/homebrew/lib/node_modules/tslab/dist/jupyter.js:375:25)


In [33]:
await game.tokenURI(0)

{"name" : "Player", "description" : "Player Badge with FiveOutOfNine#0", "image_data" : "", "attributes" : [{"trait_type" : "team", "value" : "ZERO"}, {"trait_type" : "weight", "value" : 0}, {"trait_type" : "score", "value" : 0}, {"trait_type" : "vote", "value" : "0x0000000000000000000000000000000000000000"}, {"trait_type" : "bonded_token", "value" : "0"}]}


In [34]:
JSON.parse(await game.tokenURI(1))

{
  name: 'Player',
  description: 'Player Badge with FiveOutOfNine#0',
  image_data: '',
  attributes: [
    { trait_type: 'team', value: 'ZERO' },
    { trait_type: 'weight', value: 0 },
    { trait_type: 'score', value: 0 },
    {
      trait_type: 'vote',
      value: '0x0000000000000000000000000000000000000000'
    },
    { trait_type: 'bonded_token', value: '0' }
  ]
}


In [35]:
await game.unbondLiquidity()

Error: VM Exception while processing transaction: reverted with an unrecognized custom error
    at <UnrecognizedContract>.<unknown> (0x3d63c50ad04dd5ae394cab562b7691dd5de7cf6f)
    at HardhatNode._gatherTraces (/Users/wei3erHase/wonderland/projects/buttplug-wars/node_modules/hardhat/src/internal/hardhat-network/provider/node.ts:1680:30)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)
    at HardhatNode._mineBlockWithPendingTxs (/Users/wei3erHase/wonderland/projects/buttplug-wars/node_modules/hardhat/src/internal/hardhat-network/provider/node.ts:1805:23)
    at HardhatNode.mineBlock (/Users/wei3erHase/wonderland/projects/buttplug-wars/node_modules/hardhat/src/internal/hardhat-network/provider/node.ts:494:16)
    at EthModule._sendTransactionAndReturnHash (/Users/wei3erHase/wonderland/projects/buttplug-wars/node_modules/hardhat/src/internal/hardhat-network/provider/modules/eth.ts:1522:18)
    at HardhatNetworkProvider.request (/Users/wei3erHase/wonderland/proje

In [44]:
await game.claimPrize('0x0800000000000002')

{
  hash: '0xc89b654c9f7724d46c5d8954879380adb639edf31cf1d98af2d763f410794022',
  type: 2,
  accessList: [],
  blockHash: '0x3921bc49fcb7fff11c138175bd5e1edb178ec159eb04c27321dbd3f3ab870b67',
  blockNumber: 16000048,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x3d272f4e', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x3eb3949c', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad558', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 363,
  data: '0xd70981540000000000000000000000000000000000000000000000000800000000000002',
  r: '0xe32c68dfbf7af42fad3e34dba9a51ad642197ab8e06d84fcc5baf86eb088bc83',
  s: '0x6756fdca27aeb94596a1c2f5828be813073b84e56ab6b92a8ab91b21164dc00c',
  v: 1,
  creates: null,
  chainId: 31337,
  wa

In [45]:
await evm.advanceTimeAndBlock(14*86400)
await game.withdrawLiquidity()

{
  hash: '0x4a64636b61e5be704e793560f11d754505e7a4cbc7ef69e8e1302512ba63a818',
  type: 2,
  accessList: [],
  blockHash: '0xe06a5cdc92a72941fce38af1db9eaebb3063980babcba29c32f0ac1646ca3965',
  blockNumber: 16000050,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x3cca9550', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x3dfa60a0', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 364,
  data: '0x7ea382c1',
  r: '0xc8d5661d06ca7c1573672eca2afaa9e052d5a8dcb8518f19bb7dc8d2553e4023',
  s: '0x06e95d2090880d19b332f0873bd7dcc700e3d483c74161c8aed382b8e65e2180',
  v: 1,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}


In [46]:
await game.withdrawPrize()

{
  hash: '0x13c8dbf42250d60cbba80171796bee55dbde02802e4c316afd54c9a6ad8e8781',
  type: 2,
  accessList: [],
  blockHash: '0x4ef50dc4ebb6c7786409c8ef48ed67a9c171077100c4521755b8df400fe4efa2',
  blockNumber: 16000051,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x3ca4ca81', _isBigNumber: true },
  maxPriorityFeePerGas: BigNumber { _hex: '0x3b9aca00', _isBigNumber: true },
  maxFeePerGas: BigNumber { _hex: '0x3daecb02', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x3D63c50AD04DD5aE394CAB562b7691DD5de7CF6f',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 365,
  data: '0x48d37a58',
  r: '0xb8d27294c1e2b5676387f1491ef3d45f293d31604d0499dd644f483dd7d04788',
  s: '0x043ee089cbedeab2489d00c02ab5481635fb80d95cfa2a89f350e72817c53d07',
  v: 1,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}


In [66]:
1e9/0.05e9

20


In [68]:
(1e9**3) / (0.05e9**3)

8000


unknown msg_type: comm_open
unknown msg_type: comm_msg
unknown msg_type: comm_open
unknown msg_type: comm_msg


In [63]:
0.5e9**3/1e18

125000000
